In [1]:
import pandas as pd

# EXPERIMENT DESIGN

## SOURCES INGESTION

-----------------------------------------------------------------------------------------

# EXPERIMENT RESULTS

## SOURCES INGESTION

## DATA CLEANING

## METRICS DEFINITION

In [ ]:
# calculate the metrics, use proportion_ or continuous_


## DATA ANALYSIS

In [ ]:
# select metrics and columns involved in the test

### NUMBER OF VARIANTS

In [ ]:
# if number of variants > 2

### SAMPLE SIZES

In [ ]:
# check if the sample size is large enough
# check if the variant sizes ar equal or not and the proportion

### SAMPLE DISTRIBUTION

In [ ]:
# check if the distribution is normal or not

# add q plots, histogram and violin

### SAMPLE VARIANCES

In [ ]:
# check if the variances are similar between variantes
# add boxplot

## STATISTICAL TESTING

### TEST AND TECHNICHES SELECTION

In [17]:
## Two variants tests
use_mann_whitney_u_test = False # continuous, not normal, works with unbalanced samples
use_welchs_t_test = False # continuous, normal, different variances, works with unbalanced samples (if low unbalance applied although same variants)
use_t_test = False # continuous, normal, equal variances, not recommended if unbalanced
use_z_test = False # continuous data, large sample size, known population real variance, not recommended if unbalanced, not used because you never know the real variance
use_fisher_exact_test = False # proportions, small sample size, works with unbalanced samples, # for proportions, only size matters due to central limit theorem
use_two_proportion_z_test = False # proportions, large sample size, works with unbalanced samples, used alone or post Pearson Chi-square with Bonferroni for multiple variants, # for proportions, only size matters due to central limit theorem

## Multiple variants tests
use_anova_test = False # continuous, normal, not recommended if unbalanced
use_welch_anova_test = False # continuous, normal, if unbalanced
use_pearson_chi_square_test = False # proportions, works with unbalanced samples but needs correction
use_kruskal_wallis_test = False # continuous, not normal, works with unbalanced samples

## Multiple variants post-pairs tests
use_tukey_hsd_test = False # post anova, continuous, not recommended if unbalanced
use_games_howell_test = False # post anova, continuous, for unbalanced samples
use_dunn_test = False # post kruskal wallis, continuous, needs bonferroni, works with unbalanced samples

## Multiple variants correction
use_bonferroni_correction = False # if more than 2 variants and tukey pr games howell is not used

## Unbalance data: N = A/B, if N < 2.5 use normal testing, if N < 5x use balance resampling, if N > 5x use bootstraping
use_balance_resampling = False # for unbalanced sample sizes, equalizing groups, downsampling,
use_bootstraping = False # for small sample sizes or unbalanced groups, estimating confidence intervals, upsampling,

## Additional techniques
use_bayesian_test = False # for probabilistic interpretation, alternative to p-values, small samples
use_permutation_test = False # for distribution-free significance testing, alternative to t-tests or z-tests, extrange distributions

In [20]:
## Segmentation
use_welchs_t_test_segmentation = False # for comparing two segments (A-New vs. B-New), continuous, normal, can unbalanced
use_mann_whitney_u_test_segmentation = False # for comparing two segments (A-New vs. B-New), continuous, not normal, can unbalanced
use_two_proportion_z_test_segmentation = False # for comparing two segments (A-New vs. B-New), proportions, large sample
use_fisher_exact_test_segmentation = False # for comparing two segments (A-New vs. B-New), proportions, small sample

## Interaction Tests, if discrepancies between segments
use_anova_interaction_test_segmentation = False # to test interaction effect between variant and segment, continuous, normal
use_welch_anova_interaction_test_segmentation = False # to test interaction effect between variant and segment, continuous, normal, can unbalanced
use_kruskal_wallis_interaction_test_segmentation = False # to test interaction effect between variant and segment, continuous, not normal
use_logistic_regression_interaction_test_segmentation = False # to test interaction effect for proportions, equivalent to ANOVA for categorical data
# if true, post hoc with tukey, games howell or dunn

### UNBALANCE DATA

#### BALANCE RESAMPLING

#### BOOTSTRAPING

### ADDITIONAL TECHNIQUES

#### BAYESIAN TEST

#### PERMUTATION TEST